<a href="https://colab.research.google.com/github/Yanzhi-002/text-analysis-final/blob/main/final_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Keep pandas
!pip -q install --upgrade nltk

import pandas as pd
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer

# Download the VADER lexicon (only needs to run once per runtime)
nltk.download('vader_lexicon')

sia = SentimentIntensityAnalyzer()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 21.0 MB/s eta 0:00:00


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


In [2]:
import os
from getpass import getpass

# Paste your API key when prompted (input will be hidden in Colab)
os.environ["YOUTUBE_API_KEY"] = getpass("Paste your API Key: ")

# Quick sanity check
assert os.environ.get("YOUTUBE_API_KEY"), "API key not set — please run the cell and paste your key."


Paste your API Key: ··········


In [3]:
!pip -q install requests tqdm

import os
import json
from urllib.parse import urlencode

import requests
import pandas as pd
from tqdm import tqdm

In [6]:
#搜索 AI fear 相关视频（可以改 QUERY）
API_KEY = os.environ.get("YOUTUBE_API_KEY")
BASE_URL = "https://www.googleapis.com/youtube/v3"

if not API_KEY:
    raise ValueError("Missing API key. Set os.environ['YOUTUBE_API_KEY'] first.")

def yt_get(resource: str, params: dict) -> dict:
    """Call YouTube Data API v3.
    - resource: e.g., 'search', 'videos', 'commentThreads'
    - params: dict of query params (we append the API key here)
    Returns parsed JSON as a Python dict.
    """
    q = {**params, "key": API_KEY}
    url = f"{BASE_URL}/{resource}?{urlencode(q)}"
    r = requests.get(url, timeout=30)
    r.raise_for_status()  # raise an HTTPError if the request failed
    return r.json()

In [7]:
QUERY = "AI fear"
TARGET_VIDEOS = 60
MAX_RESULTS = 50

video_hits = []
page_token = None

with tqdm(total=TARGET_VIDEOS, desc="Searching videos") as pbar:
    while len(video_hits) < TARGET_VIDEOS:
        # The 'search' resource finds videos; we request snippet data (title, channel, publishedAt).
        params = {
            "part": "snippet",
            "q": QUERY,
            "type": "video",
            "maxResults": MAX_RESULTS,
            "order": "relevance",
        }
        if page_token:
            params["pageToken"] = page_token

        data = yt_get("search", params)
        items = data.get("items", [])

        for it in items:
            vid = it.get("id", {}).get("videoId")
            if not vid:
                continue
            snip = it.get("snippet", {})
            video_hits.append({
                "video_id": vid,
                "publishedAt": snip.get("publishedAt"),
                "title": snip.get("title"),
                "channelId": snip.get("channelId"),
                "channelTitle": snip.get("channelTitle"),
            })
            pbar.update(1)
            if len(video_hits) >= TARGET_VIDEOS:
                break

        page_token = data.get("nextPageToken")
        if not page_token:
            break

videos_df = pd.DataFrame(video_hits)
videos_df.head(3)


Searching videos: 100%|██████████| 60/60 [00:00<00:00, 74.84it/s]


,video_id,publishedAt,title,channelId,channelTitle
0,9fogzZ_6kR0,2024-10-26T20:00:07Z,Is Generative AI an Existential Threat to the ...,UChirEOpgFCupRAk5etXqPaA,Bloomberg News
1,guZ-ZPfjqu0,2025-07-22T18:30:55Z,&quot;This terrifies me&quot; - OpenAI CEO on ...,UCJg9wBPyKMNA5sRDnvzmkdg,LiveNOW from FOX
2,p6QudHe8-tU,2023-08-26T15:00:30Z,Why you should NOT fear A.I.,UCctXZhXmG-kf3tlIXgVZUlw,GaryVee


In [9]:
#3. 获取视频详情（viewCount / likeCount / commentCount）
# ============================================================

def chunked(seq, size):
    for i in range(0, len(seq), size):
        yield seq[i:i+size]


video_ids = videos_df["video_id"].dropna().unique().tolist()
video_details = []

for batch in tqdm(list(chunked(video_ids, 50)), desc="Fetching video details"):
    params = {
        "part": "snippet,statistics",
        "id": ",".join(batch),
        "maxResults": 50,
    }
    data = yt_get("videos", params)
    for it in data.get("items", []):
        snip = it.get("snippet", {})
        stats = it.get("statistics", {})
        video_details.append({
            "video_id": it.get("id"),
            "title": snip.get("title"),
            "description": snip.get("description"),
            "publishedAt": snip.get("publishedAt"),
            "channelTitle": snip.get("channelTitle"),
            "viewCount": int(stats.get("viewCount", 0) or 0),
            "likeCount": int(stats.get("likeCount", 0) or 0),
            "commentCount": int(stats.get("commentCount", 0) or 0),
        })

video_details_df = pd.DataFrame(video_details)
print("Got details for videos:", len(video_details_df))
video_details_df.head(3)

Fetching video details: 100%|██████████| 2/2 [00:00<00:00, 15.59it/s]

Got details for videos: 60


,video_id,title,description,publishedAt,channelTitle,viewCount,likeCount,commentCount
0,9fogzZ_6kR0,Is Generative AI an Existential Threat to the ...,Is generative AI an existential threat to the ...,2024-10-26T20:00:07Z,Bloomberg News,326849,6490,486
1,guZ-ZPfjqu0,"""This terrifies me"" - OpenAI CEO on his 3 grea...",OpenAI CEO Sam Altman discusses his biggest wo...,2025-07-22T18:30:55Z,LiveNOW from FOX,187780,1424,464
2,p6QudHe8-tU,Why you should NOT fear A.I.,Wait until all of you learn how to prompt A.I…...,2023-08-26T15:00:30Z,GaryVee,14196,510,21


In [10]:
# 4. 获取每个视频的评论（含时间戳 & 文本）
# ============================================================

all_comments = []

for vid in tqdm(video_details_df["video_id"].tolist(), desc="Fetching comments"):
    page_token = None
    fetched = 0
    try:
        while True:
            params = {
                "part": "snippet",
                "videoId": vid,
                "maxResults": 100,  # commentThreads 每页最多 100
                "order": "time",    # 用时间顺序方便做情绪时间序列
            }
            if page_token:
                params["pageToken"] = page_token

            data = yt_get("commentThreads", params)
            items = data.get("items", [])

            if not items:
                break

            for it in items:
                top = it.get("snippet", {}).get("topLevelComment", {})
                s = top.get("snippet", {})
                all_comments.append({
                    "video_id": vid,
                    "comment_id": top.get("id"),
                    "author": s.get("authorDisplayName"),
                    "publishedAt": s.get("publishedAt"),
                    "likeCount": s.get("likeCount", 0),
                    "text": s.get("textOriginal", ""),
                })
                fetched += 1

            page_token = data.get("nextPageToken")
            if not page_token:
                break

            # 为了保护配额，可以给每个视频加一个上限，比如 500 条
            if fetched >= 500:
                break

    except requests.HTTPError as e:
        print(f"Skipping {vid} due to HTTP error: {e}")
        continue

comments_df = pd.DataFrame(all_comments)
print("Total comments fetched:", len(comments_df))
comments_df.head(3)

Fetching comments: 100%|██████████| 60/60 [00:20<00:00,  2.97it/s]

Total comments fetched: 13997


,video_id,comment_id,author,publishedAt,likeCount,text
0,9fogzZ_6kR0,UgwCnyX7eQ_55rpxxOB4AaABAg,@shashank...853,2025-11-26T17:18:12Z,0,"Should take his words seriously, as he is the ..."
1,9fogzZ_6kR0,Ugzp_k1JX1eTxJJuGlF4AaABAg,@robustbottlecap,2025-11-25T21:31:25Z,1,i suspect somehow people are propping up the t...
2,9fogzZ_6kR0,UgzWu_-ApChgxnL_R5B4AaABAg,@Universalwisdomknowledge,2025-11-25T11:13:06Z,0,The striking advantage of a cybernetic organis...


In [20]:
# 5. 第二步：轻量清洗评论文本 + 使用 VADER 计算情绪分数（compound）
# ============================================================

import re

def clean_comment(text):
    """轻量清洗，不破坏 VADER 的情绪特征"""
    if not isinstance(text, str):
        return ""
    text = text.strip()
    text = re.sub(r"\s+", " ", text)                # 删除重复空白
    text = re.sub(r"http\S+|www\S+", "", text)      # 删除网址
    text = re.sub(r"&\w+;", "", text)               # 删除 HTML 符号（如 &nbsp;）
    return text

# 使用清洗后的文本
comments_df["text_clean"] = comments_df["text"].apply(clean_comment)

# 定义情绪计算函数（原样保留）
def compound_score(text: str) -> float:
    return sia.polarity_scores(text or "")["compound"]

# 用 text_clean 来算情绪
comments_df["compound"] = comments_df["text_clean"].apply(compound_score)

# 把时间转为 datetime（后面排序会用到）
comments_df["publishedAt"] = pd.to_datetime(comments_df["publishedAt"])

comments_df.head(3)

,video_id,comment_id,author,publishedAt,likeCount,text,compound,order_in_video,text_clean
0,-KTHnRgKHSM,Ugz0uQKrwsgVjsEMp9x4AaABAg,@rlak5,2025-06-11 16:16:18+00:00,1,I'm from Bangladesh🇧🇩,0.0000,1,I'm from Bangladesh🇧🇩
1,-KTHnRgKHSM,Ugyl-z8Lu71kg1Dw0BJ4AaABAg,@prakharsingh8766,2025-06-11 16:28:05+00:00,1,These Ted talks are real gold.they have the po...,0.0772,2,These Ted talks are real gold.they have the po...
2,-KTHnRgKHSM,UgxFyYiW_kJ141ioyYV4AaABAg,@theprojectblythe,2025-06-11 16:59:38+00:00,0,I'm commenting on the first 23 secs of this vi...,0.9433,3,I'm commenting on the first 23 secs of this vi...


In [21]:
# 6. 第三步：按时间排序 + 编号，构造情绪时间序列
# ============================================================

# 按视频 & 时间排序
comments_df = comments_df.sort_values(["video_id", "publishedAt"]).reset_index(drop=True)

# 对每个视频内按时间顺序编号：1, 2, ..., N
comments_df["order_in_video"] = comments_df.groupby("video_id").cumcount() + 1

# 查看某个视频的前几条评论（测试）
example_vid = comments_df["video_id"].iloc[0]
comments_df[comments_df["video_id"] == example_vid].head(5)

,video_id,comment_id,author,publishedAt,likeCount,text,compound,order_in_video,text_clean
0,-KTHnRgKHSM,Ugz0uQKrwsgVjsEMp9x4AaABAg,@rlak5,2025-06-11 16:16:18+00:00,1,I'm from Bangladesh🇧🇩,0.0000,1,I'm from Bangladesh🇧🇩
1,-KTHnRgKHSM,Ugyl-z8Lu71kg1Dw0BJ4AaABAg,@prakharsingh8766,2025-06-11 16:28:05+00:00,1,These Ted talks are real gold.they have the po...,0.0772,2,These Ted talks are real gold.they have the po...
2,-KTHnRgKHSM,UgxFyYiW_kJ141ioyYV4AaABAg,@theprojectblythe,2025-06-11 16:59:38+00:00,0,I'm commenting on the first 23 secs of this vi...,0.9433,3,I'm commenting on the first 23 secs of this vi...
3,-KTHnRgKHSM,Ugx9U3sjvi06hyoIpvR4AaABAg,@ulrichassouto,2025-06-11 22:03:32+00:00,0,"Everybody is scared of AI. As I’m concerned, w...",-0.6656,4,"Everybody is scared of AI. As I’m concerned, w..."
4,-KTHnRgKHSM,UgzoF6GcXMuN0wLIfqp4AaABAg,@deborahann9474,2025-06-12 02:27:04+00:00,1,Not dying does not mean eternal life.,0.0000,5,Not dying does not mean eternal life.


In [22]:
# 7. 情绪回声室分析：早期 vs 后期 情绪对比（简单版）
# ============================================================

def early_late_stats(df, k=50):

    results = []
    for vid, sub in df.groupby("video_id"):
        sub = sub.sort_values("order_in_video")
        n = len(sub)
        if n < 2 * k:
            # 评论太少的视频可以跳过或用更小的 k
            continue

        early = sub.iloc[:k]
        late = sub.iloc[-k:]

        results.append({
            "video_id": vid,
            "title": video_details_df.set_index("video_id").loc[vid, "title"]
                     if vid in video_details_df["video_id"].values else None,
            "n_comments": n,
            "early_mean": early["compound"].mean(),
            "late_mean": late["compound"].mean(),
            "delta_late_minus_early": late["compound"].mean() - early["compound"].mean(),
        })
    return pd.DataFrame(results)


echo_stats = early_late_stats(comments_df, k=50)
echo_stats.sort_values("delta_late_minus_early").head(10)

,video_id,title,n_comments,early_mean,late_mean,delta_late_minus_early
15,PaOLBOuyswI,Building the AI of F.E.A.R. with Goal Oriented...,114,0.314042,0.041600,-0.272442
25,dXtwro68GiQ,"If You’re Creative But Scared About AI, Watch ...",237,0.408542,0.199782,-0.208760
31,ySqPKEfBFvY,AI is getting scary 😱 #ai,500,0.073746,-0.079216,-0.152962
6,GJeFoEw9x0M,AI CEO explains the terrifying new behavior AI...,500,-0.004410,-0.152274,-0.147864
5,EoJ6gXGL-zM,AI expert reveals his 'single greatest fear' a...,500,-0.030168,-0.165852,-0.135684
16,T2g8FF8-RLA,Sam Altman on AGI: people will freak out then ...,500,0.106636,-0.026834,-0.133470
30,qe9QSCF-d88,The Catastrophic Risks of AI — and a Safer Pat...,500,0.144154,0.026274,-0.117880
13,OG57rWI7o4o,AI boom or bubble? why economists fear a globa...,387,0.135024,0.024218,-0.110806
7,HKMb_TXvyZg,You Have No Idea How Terrified AI Scientists A...,500,0.087452,-0.011334,-0.098786
8,Hq3TtNRc0Fs,Fear and Loathing in the Age of AI,500,0.253710,0.167952,-0.085758


In [23]:
import plotly.express as px
import plotly.io as pio

pio.renderers.default = "colab"  # 确保在 Colab 里正常展示


In [24]:
# 8. 第四步：可视化——情绪折线图 & 滚动平均线（Rolling Sentiment）
# ============================================================

# 选一个评论数量比较多的视频来画图（例如评论最多的）
if not echo_stats.empty:
    # 选评论最多的一个视频
    target_video_id = echo_stats.sort_values("n_comments", ascending=False)["video_id"].iloc[0]
else:
    # 如果 echo_stats 为空（评论都太少），就选任意一个有评论的视频
    target_video_id = comments_df["video_id"].value_counts().index[0]

print("Target video for visualization:", target_video_id)

target_title = video_details_df.set_index("video_id").loc[target_video_id, "title"] \
    if target_video_id in video_details_df["video_id"].values else target_video_id

target_comments = comments_df[comments_df["video_id"] == target_video_id].copy()
target_comments = target_comments.sort_values("order_in_video")

# 计算滚动平均情绪（比如窗口大小为 20 条评论）
WINDOW = 20
target_comments["rolling_mean"] = (
    target_comments["compound"]
    .rolling(window=WINDOW, min_periods=1)
    .mean()
)

print("Number of comments in target video:", len(target_comments))
target_comments.head(5)


# --- 8.1 情绪折线图：每条评论的情绪 + 滚动平均线 ---

fig = px.line(
    target_comments,
    x="order_in_video",
    y="compound",
    title=f"Sentiment over time for one AI fear video\n{target_title[:80]}",
    labels={"order_in_video": "Comment order (time)", "compound": "VADER compound sentiment"},
)

# 再画一条滚动平均线（平滑趋势）
fig.add_scatter(
    x=target_comments["order_in_video"],
    y=target_comments["rolling_mean"],
    mode="lines",
    name=f"Rolling mean (window={WINDOW})"
)

fig.show()

# 保存图（可以放进报告或 GitHub）
fig.write_html("ai_fear_sentiment_timeseries.html", include_plotlyjs="cdn", full_html=True)
fig.write_image("ai_fear_sentiment_timeseries.png")


# --- 8.2 早期 vs 后期 情绪分布对比（箱线图或小提琴图） ---

K = 50  # 和上面一致，前 50 vs 后 50

sub = target_comments.copy()
if len(sub) >= 2 * K:
    sub["segment"] = "middle"
    sub.loc[sub["order_in_video"] <= K, "segment"] = "early"
    sub.loc[sub["order_in_video"] > len(sub) - K, "segment"] = "late"

    sub_seg = sub[sub["segment"].isin(["early", "late"])]

    fig_box = px.violin(
        sub_seg,
        x="segment",
        y="compound",
        box=True,
        points="all",
        title=f"Early vs Late comment sentiment for one AI fear video\n{target_title[:80]}",
        labels={"segment": "Comment segment", "compound": "VADER compound sentiment"},
    )
    fig_box.show()
    fig_box.write_html("ai_fear_early_vs_late_sentiment.html", include_plotlyjs="cdn", full_html=True)
    fig_box.write_image("ai_fear_early_vs_late_sentiment.png")
else:
    print("Target video has too few comments for early/late comparison.")


Target video for visualization: 7wfAcn9BJ9I
Number of comments in target video: 500


ValueError: 
Image export using the "kaleido" engine requires the kaleido package,
which can be installed using pip:
    $ pip install -U kaleido
